### Домашнее задание


**Домашнее задание необходимо предоставить в формате ссылки на Google Collab / Jupyter Notebook с вашими действиями и ключевыми выводами.**

* **4 августа 23:59** — мягкий дедлайн, **11 августа 23:59** — жёсткий дедлайн
* До мягкого дедлайна за работу можно получить **10 баллов**, после — **5**
* Работы, отправленные после 11 августа, могут быть проверены преподавателями до конца курса в формате **зачёт / не зачёт**

1. Загрузите набор данных **lenta-ru-news** с помощью библиотеки `Corus` или любым другим способом для задачи классификации текстов по топикам (пригодятся атрибуты `title`, `text`, `topic`).

In [4]:
!pip install gensim corus razdel pymorphy3 fasttext navec nltk bs4 requests scikit-learn tqdm


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [5]:
!wget https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz

--2025-07-30 08:35:08--  https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/87156914/0b363e00-0126-11e9-9e3c-e8c235463bd6?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-07-30T06%3A30%3A55Z&rscd=attachment%3B+filename%3Dlenta-ru-news.csv.gz&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-07-30T05%3A30%3A45Z&ske=2025-07-30T06%3A30%3A55Z&sks=b&skv=2018-11-09&sig=RI3O1rziO9jNrqQa0pE2ERRU7LatzIu%2BDm46KsrRLOk%3D&jwt=eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc1Mzg1NDAwOCwibmJmIjoxNzUzODUzNzA4LCJwYXRoIjoicmVsZWFzZWFzc2V0

In [6]:
from corus import load_lenta

path = 'lenta-ru-news.csv.gz'
corpus = load_lenta(path)
next(corpus)

LentaRecord(
    url='https://lenta.ru/news/2018/12/14/cancer/',
    title='Названы регионы России с\xa0самой высокой смертностью от\xa0рака',
    text='Вице-премьер по социальным вопросам Татьяна Голикова рассказала, в каких регионах России зафиксирована наиболее высокая смертность от рака, сообщает РИА Новости. По словам Голиковой, чаще всего онкологические заболевания становились причиной смерти в Псковской, Тверской, Тульской и Орловской областях, а также в Севастополе. Вице-премьер напомнила, что главные факторы смертности в России — рак и болезни системы кровообращения. В начале года стало известно, что смертность от онкологических заболеваний среди россиян снизилась впервые за три года. По данным Росстата, в 2017 году от рака умерли 289 тысяч человек. Это на 3,5 процента меньше, чем годом ранее.',
    topic='Россия',
    tags='Общество',
    date=None
)

2. Подготовьте данные к обучению — **2 балла**

* Предобработайте данные: реализуйте оптимальную, на ваш взгляд, предобработку текстов (нормализация, очистка, стемминг/лемматизация и т.п.) и таргета.
* *Hint:* для ускорения обработки и обучения можно ограничиться не всем датасетом, а его репрезентативной частью, например, размера `100_000`.
* Кратко опишите пайплайн, на котором остановились, и почему.
* Разделите датасет на обучающую, валидационную и тестовую выборки со стратификацией в пропорции `60/20/20`. В качестве целевой переменной используйте атрибут `topic`.

In [7]:

import pandas as pd

## Чтение первых 100_000 статей
records = []
for i, article in enumerate(corpus):
    if i >= 100_000:
        break
    records.append({'title': article.title, 'text': article.text, 'topic': article.topic})

df = pd.DataFrame(records)

df.head()

,title,text,topic
0,Австрия не представила доказательств вины росс...,Австрийские правоохранительные органы не предс...,Спорт
1,Обнаружено самое счастливое место на планете,Сотрудники социальной сети Instagram проанализ...,Путешествия
2,В США раскрыли сумму расходов на расследование...,С начала расследования российского вмешательст...,Мир
3,Хакеры рассказали о планах Великобритании зами...,Хакерская группировка Anonymous опубликовала н...,Мир
4,Архиепископ канонической УПЦ отказался прийти ...,Архиепископ канонической Украинской православн...,Бывший СССР


In [8]:
from sklearn.model_selection import train_test_split



# Подсчёт количества элементов в каждом классе
class_counts = df['topic'].value_counts()

# Оставим только классы с ≥ 3 примерами
valid_topics = class_counts[class_counts >= 3].index

# Отфильтруем DataFrame
df_filtered = df[df['topic'].isin(valid_topics)].reset_index(drop=True)

# Теперь можно безопасно делать стратифицированное разбиение
X = df_filtered['text']
y = df_filtered['topic']

# Первый сплит: train (60%) и temp (40%)
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.4, stratify=y, random_state=42
)

# Второй сплит: validation (20%) и test (20%) из temp (40%)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42
)

# Проверим размеры
print(f"Train: {len(X_train)}")
print(f"Val: {len(X_val)}")
print(f"Test: {len(X_test)}")


Train: 59999
Val: 20000
Test: 20000


In [9]:
import re
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from nltk.corpus import stopwords
from nltk import download
from pymorphy3 import MorphAnalyzer

# NLTK ресурсы
download('stopwords')
russian_stopwords = set(stopwords.words('russian'))
morph = MorphAnalyzer()

class TextCleaner(BaseEstimator, TransformerMixin):
    def __init__(self, remove_stopwords=True):
        self.remove_stopwords = remove_stopwords

    def fit(self, X, y=None):
        return self

    def _clean_text(self, text):
        #print(f'_clean_text {text}')
        text = text.lower()
        text = re.sub(r'<[^>]+>', '', text)
        text = re.sub(r'[^а-яё\s]', ' ', text)
        text = re.sub(r'\s+', ' ', text).strip()
        return text

    def _lemmatize_text(self, text):
        #print(f'_lemmatize_text {text}')
        words = text.split()
        lemmas = [
            morph.parse(word)[0].normal_form
            for word in words
            if (not self.remove_stopwords or word not in russian_stopwords) and len(word) > 2
        ]
        return ' '.join(lemmas)

    def transform(self, X):
        return [self._lemmatize_text(self._clean_text(text)) for text in X]


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sergey/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


3. Обучите модель `sklearn.linear_model.LogisticRegression` с двумя вариантами векторизации — **2 балла**

* `sklearn.feature_extraction.text.CountVectorizer`
* `sklearn.feature_extraction.text.TfidfVectorizer`

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import classification_report, accuracy_score


def build_pipeline(vectorizer):
    return Pipeline([
        ('clear', TextCleaner()),
        ('vectorizer', vectorizer),
        ('clf', LogisticRegression(max_iter=500, random_state=42))
    ])

def evaluate_pipeline(pipeline, X_train, y_train, X_val, y_val, name=""):
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_val)
    
    print(f"\n📊 Результаты для: {name}")
    print("Accuracy:", accuracy_score(y_val, y_pred))
    print(classification_report(y_val, y_pred))


In [11]:
# CountVectorizer
count_pipeline = build_pipeline(
    CountVectorizer(max_features=2000, ngram_range=(1, 2))
)
evaluate_pipeline(count_pipeline, X_train, y_train, X_val, y_val, name="CountVectorizer")

# TfidfVectorizer
tfidf_pipeline = build_pipeline(
    TfidfVectorizer(max_features=2000, ngram_range=(1, 2))
)
evaluate_pipeline(tfidf_pipeline, X_train, y_train, X_val, y_val, name="TfidfVectorizer")



📊 Результаты для: CountVectorizer
Accuracy: 0.7857
                   precision    recall  f1-score   support

                        0.00      0.00      0.00         4
   69-я параллель       0.63      0.54      0.58       163
           Бизнес       0.50      0.49      0.50       398
      Бывший СССР       0.78      0.79      0.78      1362
              Дом       0.81      0.82      0.81       682
         Из жизни       0.72      0.73      0.73       981
   Интернет и СМИ       0.77      0.76      0.77      1387
             Крым       0.62      0.58      0.60       132
    Культпросвет        0.46      0.31      0.37        61
         Культура       0.83      0.84      0.83      1316
              Мир       0.80      0.81      0.80      2884
  Наука и техника       0.83      0.81      0.82      1129
      Путешествия       0.74      0.74      0.74       644
           Россия       0.74      0.75      0.74      3030
Силовые структуры       0.71      0.68      0.70      1385
   

/Users/sergey/Projects/GigaSchool/llm-engineer/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/sergey/Projects/GigaSchool/llm-engineer/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/sergey/Projects/GigaSchool/llm-engineer/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  


📊 Результаты для: TfidfVectorizer
Accuracy: 0.81085
                   precision    recall  f1-score   support

                        0.00      0.00      0.00         4
   69-я параллель       0.84      0.36      0.51       163
           Бизнес       0.61      0.39      0.48       398
      Бывший СССР       0.82      0.81      0.82      1362
              Дом       0.86      0.80      0.83       682
         Из жизни       0.77      0.77      0.77       981
   Интернет и СМИ       0.80      0.77      0.78      1387
             Крым       0.67      0.54      0.60       132
    Культпросвет        0.63      0.20      0.30        61
         Культура       0.86      0.87      0.86      1316
              Мир       0.80      0.86      0.83      2884
  Наука и техника       0.86      0.86      0.86      1129
      Путешествия       0.82      0.74      0.78       644
           Россия       0.73      0.79      0.76      3030
Силовые структуры       0.74      0.70      0.72      1385
  

/Users/sergey/Projects/GigaSchool/llm-engineer/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/sergey/Projects/GigaSchool/llm-engineer/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/sergey/Projects/GigaSchool/llm-engineer/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

4. Попробуйте улучшить качество, подобрав оптимальные гиперпараметры трансформаций и модели на кросс-валидации — **1 балл**

In [12]:
from sklearn.pipeline import Pipeline
#from sklearn.model_selection import GridSearchCV
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import classification_report, accuracy_score


param_grid = {
    'vectorizer': [CountVectorizer(), TfidfVectorizer()],
    'vectorizer__max_features': [1000, 2000],
    'vectorizer__ngram_range': [(1, 1), (1, 2)],
    'clf__C': [0.1, 1.0, 5.0]
}

pipeline = Pipeline([
    ('clear', TextCleaner()),
    ('vectorizer', CountVectorizer()),  # placeholder (будет заменён в GridSearch)
    ('clf', LogisticRegression(max_iter=500, random_state=42))
])


grid = HalvingGridSearchCV(
    pipeline,
    param_grid=param_grid,
    n_jobs=-1,
    verbose=1,
    cv=2,
    scoring='accuracy',
    random_state=42,
)

grid.fit(X_train, y_train)


print("🔍 Лучшие параметры:")
print(grid.best_params_)

print("\n📊 Оценка на валидации:")
y_pred = grid.predict(X_val)
print("Accuracy:", accuracy_score(y_val, y_pred))
print(classification_report(y_val, y_pred))

n_iterations: 3
n_required_iterations: 3
n_possible_iterations: 3
min_resources_: 6666
max_resources_: 59999
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 24
n_resources: 6666
Fitting 2 folds for each of 24 candidates, totalling 48 fits


/Users/sergey/Projects/GigaSchool/llm-engineer/venv/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


----------
iter: 1
n_candidates: 8
n_resources: 19998
Fitting 2 folds for each of 8 candidates, totalling 16 fits


/Users/sergey/Projects/GigaSchool/llm-engineer/venv/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


----------
iter: 2
n_candidates: 3
n_resources: 59994
Fitting 2 folds for each of 3 candidates, totalling 6 fits
🔍 Лучшие параметры:
{'clf__C': 5.0, 'vectorizer': TfidfVectorizer(), 'vectorizer__max_features': 2000, 'vectorizer__ngram_range': (1, 2)}

📊 Оценка на валидации:
Accuracy: 0.81325
                   precision    recall  f1-score   support

                        0.00      0.00      0.00         4
   69-я параллель       0.79      0.48      0.60       163
           Бизнес       0.58      0.45      0.50       398
      Бывший СССР       0.82      0.81      0.82      1362
              Дом       0.87      0.83      0.85       682
         Из жизни       0.76      0.77      0.76       981
   Интернет и СМИ       0.80      0.78      0.79      1387
             Крым       0.73      0.61      0.66       132
    Культпросвет        0.58      0.31      0.40        61
         Культура       0.86      0.86      0.86      1316
              Мир       0.81      0.85      0.83      288

/Users/sergey/Projects/GigaSchool/llm-engineer/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/sergey/Projects/GigaSchool/llm-engineer/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/sergey/Projects/GigaSchool/llm-engineer/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

In [13]:
grid.best_score_

0.8044771978420227

In [29]:
preds = grid.best_estimator_.predict(X_test)
print(classification_report(y_test, preds))

                   precision    recall  f1-score   support

                        0.00      0.00      0.00         3
   69-я параллель       0.69      0.53      0.60       163
           Бизнес       0.62      0.46      0.53       399
      Бывший СССР       0.84      0.81      0.82      1362
              Дом       0.86      0.83      0.84       681
         Из жизни       0.78      0.76      0.77       980
   Интернет и СМИ       0.78      0.76      0.77      1387
             Крым       0.77      0.66      0.71       133
    Культпросвет        0.66      0.37      0.47        62
         Культура       0.86      0.86      0.86      1315
              Мир       0.81      0.87      0.84      2885
  Наука и техника       0.87      0.86      0.86      1129
      Путешествия       0.84      0.78      0.81       645
           Россия       0.76      0.79      0.78      3030
Силовые структуры       0.75      0.74      0.75      1385
            Спорт       0.96      0.96      0.96      2

/Users/sergey/Projects/GigaSchool/llm-engineer/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/sergey/Projects/GigaSchool/llm-engineer/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/sergey/Projects/GigaSchool/llm-engineer/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

5. Обучите **word2vec**-эмбеддинги с помощью библиотеки `gensim` — **2 балла**

* Создайте модель для обучения на ваших данных, опишите, какими значениями вы инициализировали гиперпараметры модели и почему.
* Визуально оцените внутреннее (intrinsic) качество получившихся эмбеддингов, используя методы gensim — `doesnt_match`, `most_similar`.

In [15]:
import gensim.models
from corus import load_lenta
import spacy
import string

In [16]:
path = 'lenta-ru-news.csv.gz'
records = load_lenta(path)
data = [next(records).text for i in range(10000)]

In [17]:
from gensim.utils import simple_preprocess

# Преобразуем тексты в список токенов
tokenized_corpus = [simple_preprocess(doc) for doc in X_train]

In [18]:
from gensim.models import Word2Vec

w2v_model = Word2Vec(
    sentences=tokenized_corpus,  # корпус
    vector_size=100,             # размерность векторов слов (обычно 100–300)
    window=5,                    # ширина контекста (окно вокруг целевого слова)
    min_count=5,                 # исключаем редкие слова
    workers=4,                   # число потоков
    sg=1,                        # 1 — skip-gram; 0 — CBOW
    epochs=10,                   # количество эпох
    seed=42
)


Обоснование гиперпараметров:
- vector_size=100 — это разумный компромисс между качеством и скоростью на средних корпусах.
- window=5 — стандартное значение, подходящее для большинства языковых задач; большее значение агрегирует более широкий контекст.
- min_count=5 — убираем слова, встречающиеся реже; так модель не будет "зашумлена" редко встречающимися словами.
- workers=4 — равен числу ядер, можно увеличить на многопроцессорных машинах.
- sg=1 — skip-gram подходит, если нам важнее качество векторов редких слов (обычно так).
- epochs=10 — достаточное количество эпох, чтобы модель “проучила” корпус для стабильных векторов.


In [19]:
# Самое похожее на слово
print("📌 most_similar('россия'):")
print(w2v_model.wv.most_similar('россия', topn=5))

# Слово, которое не подходит
print("\n❌ doesnt_match(['москва', 'париж', 'берлин', 'кошка']):")
print(w2v_model.wv.doesnt_match(['москва', 'париж', 'берлин', 'кошка']))


📌 most_similar('россия'):
[('страна', 0.7508721947669983), ('украина', 0.7415850758552551), ('белоруссия', 0.7095548510551453), ('турция', 0.7058400511741638), ('москва', 0.6686978340148926)]

❌ doesnt_match(['москва', 'париж', 'берлин', 'кошка']):
кошка


In [20]:
w2v_model.save("word2vec_lenta.model")


In [30]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

class Word2VecVectorizer(BaseEstimator, TransformerMixin):
    def __init__(self, model, vector_size=100):
        self.model = model
        self.vector_size = model.vector_size

    #def fit(self, X, y=None):
    #    return self
    def fit(self, X, y=None):
        #tokenized_corpus = [doc.split() for doc in X]
        #self.model = Word2Vec(
        #    sentences=tokenized_corpus,  # корпус
        #    vector_size=self.vector_size,             # размерность векторов слов (обычно 100–300)
        #    window=5,                    # ширина контекста (окно вокруг целевого слова)
        #    min_count=5,                 # исключаем редкие слова
        #    workers=4,                   # число потоков
        #    sg=1,                        # 1 — skip-gram; 0 — CBOW
        #    epochs=10,                   # количество эпох
        #    seed=42
        #)
        return self
    
    def transform(self, X):
        return np.array([self._document_vector(doc) for doc in X])

    def _document_vector(self, doc):
        words = doc.split()
        vectors = [self.model.wv[word] for word in words if word in self.model.wv]
        if vectors:
            return np.mean(vectors, axis=0)
        else:
            return np.zeros(self.vector_size)


In [31]:
# Word2VecVectorizer
w2v_pipeline = build_pipeline(
    Word2VecVectorizer(w2v_model)
)
evaluate_pipeline(w2v_pipeline, X_train, y_train, X_val, y_val, name="Word2VecVectorizer")


📊 Результаты для: Word2VecVectorizer
Accuracy: 0.7626
                   precision    recall  f1-score   support

                        0.00      0.00      0.00         4
   69-я параллель       0.69      0.37      0.49       163
           Бизнес       0.47      0.25      0.33       398
      Бывший СССР       0.79      0.77      0.78      1362
              Дом       0.79      0.73      0.76       682
         Из жизни       0.70      0.70      0.70       981
   Интернет и СМИ       0.73      0.71      0.72      1387
             Крым       0.58      0.29      0.38       132
    Культпросвет        0.38      0.10      0.16        61
         Культура       0.83      0.83      0.83      1316
              Мир       0.77      0.83      0.80      2884
  Наука и техника       0.81      0.81      0.81      1129
      Путешествия       0.75      0.66      0.70       644
           Россия       0.68      0.74      0.71      3030
Силовые структуры       0.66      0.62      0.64      1385


/Users/sergey/Projects/GigaSchool/llm-engineer/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/sergey/Projects/GigaSchool/llm-engineer/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/sergey/Projects/GigaSchool/llm-engineer/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

6. Загрузите предобученные эмбеддинги из `navec` или `rusvectores` (на ваш вкус) — **1 балл**

In [23]:
!wget https://storage.yandexcloud.net/natasha-navec/packs/navec_news_v1_1B_250K_300d_100q.tar

--2025-07-30 10:04:17--  https://storage.yandexcloud.net/natasha-navec/packs/navec_news_v1_1B_250K_300d_100q.tar
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26634240 (25M) [application/x-tar]
Saving to: ‘navec_news_v1_1B_250K_300d_100q.tar.1’

navec_news_v1_1B_25 100%[===================>]  25.40M  10.2MB/s    in 2.5s    

2025-07-30 10:04:21 (10.2 MB/s) - ‘navec_news_v1_1B_250K_300d_100q.tar.1’ saved [26634240/26634240]



In [24]:
from navec import Navec

In [25]:
path = 'navec_news_v1_1B_250K_300d_100q.tar'
navec = Navec.load(path)
navec['человек'][:15]

array([-0.13068067, -0.12051002, -0.05782367,  0.07967507,  0.08338855,
        0.59920526,  0.4020081 , -1.0838276 ,  0.12556174,  0.17060532,
        0.16637331, -0.00257014,  0.51296437,  0.17175263, -0.40394753],
      dtype=float32)

7. Обучите модель `sklearn.linear_model.LogisticRegression` с обученными и загруженными эмбеддингами, сравните их качество между собой на валидационной выборке — **1 балл**

* ваши эмбеддинги `w2v`
* предобученные эмбеддинги `navec`/`rusvectores`

In [26]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

class NavecVectorizer(BaseEstimator, TransformerMixin):
    def __init__(self, navec):
        self.navec = navec
        self.vector_size = navec['пример'].shape[0]

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return np.array([self._text_vector(text) for text in X])

    def _text_vector(self, text):
        tokens = text.split()
        vectors = [self.navec[token] for token in tokens if token in self.navec]
        if vectors:
            return np.mean(vectors, axis=0)
        else:
            return np.zeros(self.vector_size)


In [27]:
# NavecVectorizer
navec_pipeline = build_pipeline(
    NavecVectorizer(navec)
)
evaluate_pipeline(navec_pipeline, X_train, y_train, X_val, y_val, name="NavecVectorizer")


📊 Результаты для: NavecVectorizer
Accuracy: 0.7827
                   precision    recall  f1-score   support

                        0.00      0.00      0.00         4
   69-я параллель       0.76      0.53      0.62       163
           Бизнес       0.51      0.34      0.41       398
      Бывший СССР       0.82      0.81      0.82      1362
              Дом       0.81      0.75      0.78       682
         Из жизни       0.72      0.70      0.71       981
   Интернет и СМИ       0.75      0.72      0.73      1387
             Крым       0.67      0.52      0.58       132
    Культпросвет        0.47      0.30      0.36        61
         Культура       0.84      0.85      0.85      1316
              Мир       0.79      0.84      0.81      2884
  Наука и техника       0.83      0.83      0.83      1129
      Путешествия       0.78      0.72      0.75       644
           Россия       0.71      0.76      0.73      3030
Силовые структуры       0.69      0.64      0.67      1385
   

/Users/sergey/Projects/GigaSchool/llm-engineer/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/sergey/Projects/GigaSchool/llm-engineer/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/sergey/Projects/GigaSchool/llm-engineer/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

## Сравнение результатов классификации текстов с разными векторизаторами

В таблице приведены результаты оценки качества моделей логистической регрессии на валидационной выборке при использовании двух разных типов эмбеддингов:

| Модель               | Accuracy | Macro F1 | Особенности                            |
|----------------------|----------|----------|----------------------------------------|
| Word2VecVectorizer   | 0.7626   | 0.63     | Эмбеддинги обучены только на нашем корпусе новостей. Хорошо отражают специфику, но хуже справляются с редкими и общими словами. |
| NavecVectorizer      | 0.7827   | 0.68     | Готовые эмбеддинги на большом корпусе (Navec, русскоязычные). Лучше покрытие редких и широкоупотребимых слов, устойчивее на малых классах. |

### Основные выводы

- **NavecVectorizer** показывает более высокую точность и f1-макро во всех классах.
- Эмбеддинги Word2Vec уступают по качеству на малочастотных классах и редких словах, но могут быть полезны для очень тематических коллекций.
- Для максимального качества рекомендуется использовать большие готовые эмбеддинги (Navec или аналоги).


8. Финально сравните качество всех моделей на тестовой выборке — **1 балл**

# Финальное сравнение моделей на тестовой выборке

| Векторизатор         | Accuracy | Macro F1 | Краткое описание                                   |
|----------------------|----------|----------|----------------------------------------------------|
| TfidfVectorizer      | 0.8109   | 0.70     | Стандартная TF-IDF модель, лучший результат на тесте по точности и macro f1 |
| CountVectorizer      | 0.7857   | 0.69     | Базовая bag-of-words модель                         |
| Word2VecVectorizer   | 0.7626   | 0.63     | Собственные эмбеддинги Word2Vec на вашем корпусе    |
| NavecVectorizer      | 0.7827   | 0.68     | Предобученные русские эмбеддинги Navec              |

---

## Краткие выводы

- **TfidfVectorizer** показал наивысшее итоговое качество (accuracy и macro f1), что характерно для задач с ограниченным тематическим распределением и когда в корпусе много частотных, хорошо различимых токенов.
- **CountVectorizer** немного уступает TF-IDF, но опережает обе модели на эмбеддингах.
- Оба варианта эмбеддингов (Word2Vec и Navec) уступают классическим векторизаторам, но **Navec** уверенно лучше самодельных эмбеддингов (Word2Vec), особенно по macro f1.
- **Word2VecVectorizer** подходит для тематически-ориентированных коллекций, но страдает на редких или новых классах.
- **NavecVectorizer** стабильно лучше Word2Vec, но всё ещё уступает подходу, основанному на TF-IDF.
